## Set up

In [13]:
import sys
sys.path.append('./scripts/')

import matplotlib.pyplot as plt
import seaborn as sns
import math
import numpy as np
import copy
sns.set_style("darkgrid")
from PIL import Image
import torch

import imp
import my_datasets
import utilities 
imp.reload(my_datasets) 
imp.reload(utilities) 

<module 'utilities' from '/home/laura/sw-interpretability/scripts/utilities.py'>

In [2]:
dataset='ilsvrc12'
paths, count, y, idx_to_labels = my_datasets.get_dataset(dataset)

print(count, len(paths))

10000 10000


In [3]:
layer='Mixed_7b.cat_2'
SAVEFOLD0=f'../outputs/{dataset}'

SAVEFOLD=f"{SAVEFOLD0}/{layer}/"

In [4]:
# gradients_wrt_conv_layer=np.load(f"{SAVEFOLD}/gradients_wrt_conv_layer.npy")
predictions=np.load(f"{SAVEFOLD}/predictions.npy", mmap_mode = 'r')
conv_maps=np.load(f"{SAVEFOLD}/conv_maps.npy", mmap_mode = 'r')

pvh=np.load(f"{SAVEFOLD}/eigenvectors.npy",allow_pickle=True, mmap_mode = 'r')

In [5]:
conv_maps_avg = conv_maps.mean(3).mean(2)
activations = conv_maps_avg

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [9]:
model = torch.hub.load('pytorch/vision:v0.9.0', 'inception_v3', pretrained=True)
model.eval()
model.to(device)

Using cache found in /home/laura/.cache/torch/hub/pytorch_vision_v0.9.0
/home/laura/anaconda3/envs/sw-interpretability/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/laura/anaconda3/envs/sw-interpretability/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

## Utilities

In [20]:
# def get_preds(paths, y, model, batch_start=0, batch_size=32):
#     softmaxf = torch.nn.Softmax(dim=1)
#     p1 = np.zeros((len(y),1000))
#     while batch_start+batch_size < len(y)+batch_size: 
#             # preprocessing the inputs 
#             inputs = torch.stack([transform_normalize(transform(Image.open(paths[i]).convert("RGB"))) for i in range(batch_start, min(batch_start+batch_size, len(y)))])
#             inputs = inputs.clone().detach().requires_grad_(True)
#             batch_y=y[batch_start:min(batch_start+batch_size, len(y))]

#             # transfering to GPU
#             inputs=inputs.to(device)
#             y1=model(inputs)

#             p1[batch_start:min(batch_start+batch_size, len(y)),:]=softmaxf(y1).detach().cpu()
#             batch_start+=batch_size
#     return p1

In [27]:
# def check_acc(preds, y):
#     num_correct = 0
#     for idx in range(len(y)):
#         if np.argmax(preds[idx]) == y[idx]:
#             num_correct+=1
#     acc = num_correct / len(y)
#     return acc

## Neuron abalation

In [38]:
neurons = [57] 

In [39]:
# reference 
# p_orig = utilities.get_preds(paths, y, model)
# acc_orig = utilities.check_acc(p_orig, y)
# acc_orig

In [40]:
# model.Mixed_7c
# model.Mixed_7c.branch1x1.conv.weight.shape # torch.Size([320, 2048, 1, 1])
# model.Mixed_7c.branch3x3_1.conv.weight.shape # torch.Size([384, 2048, 1, 1])
# model.Mixed_7c.branch3x3dbl_1.conv.weight.shape # torch.Size([448, 2048, 1, 1])
# model.Mixed_7c.branch_pool.conv.weight.shape # torch.Size([192, 2048, 1, 1])

In [41]:
model_ablation = torch.hub.load('pytorch/vision:v0.9.0', 'inception_v3', pretrained=True)
model_ablation.eval()
model_ablation.to(device)
# from datasets import *


Using cache found in /home/laura/.cache/torch/hub/pytorch_vision_v0.9.0


Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [42]:
# set neurons coming into next layer from abalated neurons to zero 
with torch.no_grad():
    model_ablation.Mixed_7c.branch1x1.conv.weight[:,neurons]=0.
    model_ablation.Mixed_7c.branch3x3_1.conv.weight[:,neurons]=0.
    model_ablation.Mixed_7c.branch3x3dbl_1.conv.weight[:,neurons]=0. #####################
    model_ablation.Mixed_7c.branch_pool.conv.weight[:,neurons]=0.

In [43]:
model.Mixed_7c.branch_pool.conv.weight[:,neurons].shape # torch.Size([192, 1, 1, 1])

torch.Size([192, 1, 1, 1])

In [44]:
p_ablation = utilities.get_preds(paths, y, model_ablation)
acc_ablation = utilities.check_acc(p_ablation, y)
acc_ablation

0.9202

In [45]:
acc_orig = 0.9198
diff = acc_orig - acc_ablation
diff

-0.00040000000000006697

Accuracy on whole testset: 

acc_orig = 0.9198

neurons - [1215]; acc_ablation - 0.9157; diff - 0.0040999999999999925

neurons - [27]; acc_ablation - 0.9195; diff - 0.00029999999999996696

neurons - [35]; acc_ablation - 0.9197; diff - 9.999999999998899e-05

neurons - [57]; acc_ablation - 0.9202; diff - -0.0040999999999999925

### Accuracy on top images

In [98]:
neuron = 1215
top = 25 
top_ims = utilities.get_activations(activations_avg = conv_maps_avg, neuron = neuron).argsort()[-top:][::-1]
ppaths=[paths[i] for i in top_ims]

In [99]:
p_orig = utilities.get_preds(ppaths, y[top_ims], model)
acc_orig = utilities.check_acc(p_orig, y[top_ims])
acc_orig

0.76

In [100]:
p_ablation = utilities.get_preds(ppaths, y[top_ims], model_ablation)
acc_ablation = utilities.check_acc(p_ablation, y[top_ims])
acc_ablation

0.76

In [101]:
diff = acc_orig - acc_ablation
diff

0.0

Accuracy on top images: 

neurons - [1215]; acc_orig - 0.76; acc_ablation - 0.76; diff - 0.0

neurons - [27]; acc_orig - 0.84; acc_ablation - 0.84; diff - 0.0

neurons - [35]; acc_orig - 0.88; acc_ablation - 0.88; diff - 0.0

neurons - [57]; acc_orig - 0.92; acc_ablation - 0.92; diff - 0.0

## RW

### dataloader

https://pytorch.org/tutorials/beginner/basics/data_tutorial.html - intro to datasets and dataloader classes, boilerplate for custom dataloader. 

https://pytorch.org/tutorials/beginner/data_loading_tutorial.html - WRITING CUSTOM DATASETS, DATALOADERS AND TRANSFORMS 

https://wandb.ai/sauravmaheshkar/Dataset-DataLoader/reports/An-Introduction-to-Datasets-and-DataLoader-in-PyTorch--VmlldzoxMDI5MTY2

https://discuss.pytorch.org/t/get-file-names-and-file-path-using-pytorch-dataloader/124920 

https://stackoverflow.com/questions/68112479/get-file-names-and-file-path-using-pytorch-dataloader

In [ ]:
import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset

class CustomImageDataset(Dataset):
    def __init__(self, y, paths, transform=None, target_transform=None):
        self.y = y
        self.paths = paths
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        # img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(paths[idx])
        label = self.y[idx]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

### Check model accuracy

In [ ]:
p_orig = get_preds(paths, y, model)
# p_orig.shape # (10000, 1000)
# y.shape # (10000,)

In [ ]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
        
        print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}') 
    
check_accuracy(dataloader, model)